In [ ]:
# https://arxiv.org/pdf/2003.06505.pdf

In [ ]:
# Use MXNet kernel. If not, install MXNet too : 'mxnet' (CPU) or 'mxnet-cu92' (GPU)

In [ ]:
%%sh
pip -q install autogluon ipywidgets tqdm s3fs

In [ ]:
import pandas as pd

In [ ]:
# https://www.kaggle.com/arashnic/ctr-in-advertisement (Public domain)

dataset = pd.read_csv('Ad_click_prediction_train.csv')

In [ ]:
dataset

In [ ]:
dataset.to_csv('dataset_autopilot.csv', index=False, header=True)

In [ ]:
import sagemaker
sess = sagemaker.Session()

prefix = 'sagemaker/DEMO-autogluon-click'
dataset_path = sess.upload_data(path='dataset_autopilot.csv', key_prefix=prefix + '/input/training')
print(dataset_path)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor 

In [ ]:
# https://auto.gluon.ai/dev/api/autogluon.tabular.models.html

train_data = TabularDataset(dataset_path)
#train_data = train_data.sample(n=10000, random_state=0)

In [ ]:
from autogluon.core.features.feature_metadata import FeatureMetadata

In [ ]:
# 1st try: inferred data types
# Problem here is that many categorical features are incorrectly picked up as integers

feature_metadata = FeatureMetadata.from_df(train_data)

In [ ]:
print(feature_metadata)

In [ ]:
# 2nd try: explicit data types

feature_metadata = FeatureMetadata({
    'product_category_1':'category',
    'product_category_2':'category',
    'user_group_id':'category',
    'age_level':'category',
    'user_depth':'category',
    'city_development_index':'category',
    'session_id':'category',
    'user_id':'category',
    'campaign_id':'category',
    'webpage_id':'category',
    'gender':'object',
    'var_1':'int',
    'product':'object',
    'DateTime':'datetime64'
})

In [ ]:
print(feature_metadata)

In [ ]:
predictor = TabularPredictor(
    label='is_click', 
    eval_metric='roc_auc'
)

In [ ]:
# Comment / uncomment according to 1st, 2nd and 3rd try

predictor.fit(
    train_data, 
    time_limit=3600, 
    presets='best_quality',
    feature_metadata=feature_metadata,
    excluded_model_types=['KNN','FASTAI']
)

In [ ]:
leaderboard = predictor.leaderboard() 